# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Assignment 7: Spark Streaming


## Learning Objectives
At the end of the experiment, you will be able to :

* understand Spark Structured Streaming pipeline and implement it using Pyspark.
* understand Structured Streaming Query along with steps involved.

## Information

### Streaming data:
Streaming data is data that is continuously generated by different sources such as applications, networking devices, server log files, website activity, banking transactions and location data. Such data should be processed incrementally using Stream Processing techniques without having access to all of the data. Besides, it should be considered that concept drift may happen in the data which means that the properties of the stream may change over time.
It is usually used in the context of big data in which it is generated by many different sources at high speed.

To know more about streaming data click [here](https://aws.amazon.com/streaming-data/).


### Examples of streaming data
* IoT/Sensor generated data at certain time intervals in vehicles, industrial equipments, and farm machineries can be monitored to check performance and detect defects in advance.
* Continuous changing stock market value can be tracked for the financial institution in real-time.
* A media publisher streams billions of clickstream records from its online properties, aggregates and enriches the data with demographic information about users, and optimizes content placement on its site, delivering relevancy and a better experience to its audience.

### Setup Steps:

In [ ]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "" #@param {type:"string"}

In [ ]:
#@title Please enter your password (your registered phone number) to continue: { run: "auto", display-mode: "form" }
password = "" #@param {type:"string"}

In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()

notebook= "M7_AST_07_Spark_Streaming_C" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")
    ipython.magic("sx wget https://cdn.iisc.talentsprint.com/CDS/Datasets/TempHumi.csv")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://learn-iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError
    else:
      return Answer
  except NameError:
    print ("Please answer Question")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



#### Install Pyspark

In [ ]:
!pip -qq install pyspark

#### Importing required packages

In [ ]:
import os
import time
import random
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

#### Starting the Spark Session

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

#### Reading Data



In [ ]:
df=spark.read.csv('TempHumi.csv',header=True,inferSchema=True)

In [ ]:
df.columns

In [ ]:
df.show(2)

In [ ]:
df.select('Date').dtypes # Checking data type of any column

In [ ]:
df.groupBy("Hour").count().show(24)

We can now save the output of that job by filtering on each step and saving it to a separate file

In [ ]:
!mkdir HourFolder  # Making a folder on current directory

In [ ]:
steps = df.select("Hour").distinct().collect()

In [ ]:
print(steps)
print(type(steps))
print(steps[0])
print(steps[0][0])

In [ ]:
df_test = df.where(f"Hour={steps[0][0]}")
df_test.show()
type(df_test)

###  Creating the streaming version of this input
Implementing the above steps in the loop and making individual `csv` files for each hour step. These files are saved in a folder as given below. We will use this folder as a source of incoming stream data and, we will read each file one by one as if it is a stream.

To know more about coalesce, click [here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.coalesce.html)

In [ ]:
for step in steps:
  df1 = df.where(f"Hour={step[0]}")
  df1.coalesce(1).write.csv(path='/content/HourFolder/Hourly_Record',header="true",mode="append")

In [ ]:
Hourly_Record_list = [i for i in os.listdir("/content/HourFolder/Hourly_Record/") if i.endswith(".csv")]
Hourly_Record_list

#### Checking any one file from above :

In [ ]:
file_name = Hourly_Record_list[random.randint(0,len(Hourly_Record_list)-1)]
file_name

In [ ]:
part =spark.read.csv("/content/HourFolder/Hourly_Record/"+file_name, header=True,inferSchema=True)
part.show()

### The Programming Model of Structured Streaming
“Table” is a well-known concept that developers are familiar with when building
batch applications. Structured Streaming extends this concept to streaming applications by treating a stream as an unbounded, continuously appended table, as illustrated in Figure.

![data_stream_unbounded%20table.PNG](https://cdn.iisc.talentsprint.com/CDS/Images/data_stream_unbounded_table.PNG)

Every new record received in the data stream is like a new row being appended to the unbounded input table. This leads to a new stream processing model that is very similar to a batch processing model. We will express our streaming computation as a standard batch-like query as on a static table, and Spark runs it as an incremental query on the unbounded input table. Let’s understand this model in more detail. A query on the input will generate the “Result Table” as given in the figure below. Every trigger interval (say, every 1 second), new rows get appended to the Input Table, which eventually updates the Result Table. Whenever the result table gets updated, we would want to write the changed result rows to an external sink.

![spark%20stream.PNG](https://cdn.iisc.talentsprint.com/CDS/Images/spark_stream.PNG)

The “Output” is defined as what gets written out to the external storage. The output can be defined in a different mode:

Complete Mode - The entire updated Result Table will be written to the external storage. It is up to the storage connector to decide how to handle the writing of the entire table.

Append Mode - Only the new rows appended in the Result Table since the last trigger will be written to the external storage. This is applicable only on the queries where existing rows in the Result Table are not expected to change.

Update Mode - Only the rows that were updated in the Result Table since the last trigger will be written to the external storage (available since Spark 2.1.1). Note that this is different from the Complete Mode in that this mode only outputs the rows that have changed since the last trigger. If the query doesn’t contain aggregations, it will be equivalent to Append mode.

Note that each mode is applicable on certain types of queries.

### Five Steps to Define a Streaming Query
We have prepared our data as if it is coming from a continuous streaming source now in this section, we will explore the steps involved in defining a streaming query.
#### Step 1: Define input sources
*  The first step is to define a DataFrame from a streaming source. For streaming sources, we need spark .readStream to create a DataStreamReader.
* Apache Spark natively supports reading data streams from Apache Kafka and all the various file-based formats that DataFrameReader supports (Parquet, ORC, JSON, etc.).
* We can think of the folder containing 24 csv files as the Kafka source feeding to spark streaming.

In [ ]:
part.schema # checking the part schema

In [ ]:
streaming = (spark.readStream.schema(part.schema).option('maxFilesPerTrigger',1).csv('HourFolder/Hourly_Record/'))
# maxFilesPerTrigger: maximum number of new files to be considered in every trigger, here taken 1.
# File source - Reads files written in a directory as a stream of data, here directory: HourFolder/Hourly_Record.
# Supported file formats are text, CSV, JSON, ORC, Parquet, here is CSV.
# Files will be processed in the order of file modification time.

In [ ]:
type(streaming)

#### Step 2: Setting  Up Transformation:


In [ ]:
Hourly_Mean_Value = streaming.groupBy('Hour').mean("Temperatur (C)","Humidity (%)").orderBy(F.desc("Hour"))

* Hourly_Mean_Value is a streaming DataFrame (that is, a DataFrame on unbounded, streaming data) that represents the running mean that will be computed once the streaming query is started and the streaming input data is being continuously processed.
* Now that we have our transformation, we need to specify an output sink for the results. for this example, we are going to write to a memory sink that keeps the results in memory.
* We also need to define how spark will output that data. In this example, we will use the complete output mode (rewriting all of the keys along with their counts after every trigger).
* In this example, we will not include activity query.awaitTermination() because it is required only to prevent the driver process from terminating when the stream is active. So to be able to run this locally in a notebook we will not include it.

#### Step 3: Define output sink and output mode

In [ ]:
writer = (Hourly_Mean_Value.writeStream.queryName("Temp_Humi_Mean").format("memory").outputMode("complete").start())
# Memory sink (for debugging) - The output is stored in memory as an in-memory table.
# Both, Append and Complete output modes, are supported.
# This should be used for debugging purposes on low data volumes.
# as the entire output is collected and stored in the driver’s memory. Hence, use it with caution.
# Have all the aggregates in an in-memory table. The query name will be the table name, here: "Temp_Humi_Mean".
# Note that we have to call start() to start the execution of the query.
# This returns a StreamingQuery object which is a handle to the continuously running execution.

#### Step 4: Specifying processing details:
 * Triggering details: This indicates when to trigger the discovery and processing of newly available streaming data.
 * Checkpoint location: This option is necessary for failure recovery in the real application.

 For sake of simplicity, these parameters are not set by us here. By default trigger --> The streaming query executes data in micro-batches where the next micro-batch is triggered as soon as the previous micro-batch has been completed.


#### Step 5: Start the query :
* Once everything has been specified, the final step is to start the query, already done in the above cell by .start(). We have created a table with --> .queryName ("Temp_Humi_Mean") which is updated according to trigger.
* This is a StreamingQuery object which is a handle to the continuously running execution. We can use this object to manage the query.

In [ ]:
for x in range(45):
  print('Query Result at time step : ',x)
  df_q = spark.sql("SELECT * FROM Temp_Humi_Mean")
  df_q.show(24)
  time.sleep(1)

#### Check if the stream is active

In [ ]:
spark.streams.active  # get the list of currently active streaming queries.

In [ ]:
spark.streams.active[0].isActive

In [ ]:
writer.status
# It gives information about what the query is immediately doing - is a trigger active, is data being processed, etc.
# Will print something like the following.

If we want to turn off the stream we'll run writer.stop() to reset the query for testing purposes.

In [ ]:
writer.stop()  # stop the query.

### Please answer the questions below to complete the experiment:




In [ ]:
# @title In the transformation step, how does the groupby function operate? { run: "auto", form-width: "500px", display-mode: "form" }
Answer = "" #@param ["", "It performs groupby on each file ingested at each time step", "It performs groupby at the end of the ingestion of all the files", "It performs groupby on a table formed after appending the new file to the previous one at each time step or trigger"]

In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")